# Import

In [1]:
from dotenv import load_dotenv
import os
import json
from typing import List, Union

from src.core.trajectory import Trajectory
from src.core.messages import (
    BaseMessage,
    UserMessage,
    AIMessage,
    ToolCallMessage,
    ToolCallOutputMessage,
    ChatMessageList,
    ToolMessageList
)
from src.core.tools import TOOLS
from src.env.appworld_env import AppWorldEnv, evaluate_agent
from src.llm.openai_client import OpenAIClient
from src.agent.base import BaseAgent

from src.agent.react import ReActAgent
from src.agent.reflexion import ReflexionAgent

from openai import OpenAI

from appworld import AppWorld, load_task_ids

# Env Setup

In [2]:
env_path = os.path.join("config", ".env")

load_dotenv(env_path)

True

# OpenAI Response API

## Simple response request

In [ ]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
response = llm_client.responses.create(
    model="gpt-4o",
    temperature=0.0,
    input=[
        {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
        {'role' : 'user', 'content' : '안녕하세요 당신은 누구인가요?'}
    ]
)

# Output Parsing
if hasattr(response, "output"):
    for output in response.output:
        if hasattr(output, "content"):
            for content in output.content:
                if hasattr(content, "text"):
                    print(content.text)

안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?


In [11]:
print(response)

Response(id='resp_000b7636b507e0e900694382c64b4c81939aa2cc96dedb10a1', created_at=1766032070.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_000b7636b507e0e900694382c700908193b54878f8b067e23c', content=[ResponseOutputText(annotations=[], text='안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='di

## Tool Call


| Field | Description |
| :---: | :---: |
| type	| This should always be function |
| name	| The function's name (e.g. get_weather) |
| description	| Details on when and how to use the function |
| parameters	| JSON schema defining the function's input arguments |
| strict	| Whether to enforce strict mode for the function call |

In [4]:
TOOLS = [
    {
        "type": "function",
        "name": "get_weather",
        "description": """
        이 도구는 특정 도시의 현재 날씨를 조회하기 위해 사용합니다.
        
        이 도구를 활용하기 위한 입력 인자는 다음과 같습니다.
        <입력 인자>
        location: 도시 이름
        units: 온도 단위 (celsius, fahrenheit, None)
        </입력 인자>

        이 도구를 사용하기 위한 JSON Schema 예시는 다음과 같습니다.
        <JSON Schema>
        {
            "location": "Seoul, KR",
            "units": "celsius"
        }
        </JSON Schema>
        """,
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "예: Seoul, KR"},
                "units": {"type": ["string", "null"], "enum": ["celsius", "fahrenheit", None]},
            },
            "required": ["location", "units"],
            "additionalProperties": False,
        },
    },
]

In [3]:
def get_weather(location, units=None):
    return f"Weather in {location} is 10 {units}."

In [25]:
from typing import Literal

def tmp(a : int, b:Literal["1", "2"]):
    return a, b

In [23]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

trajectory = [
    {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
    {'role' : 'user', 'content' : '현재 서울의 날씨는 어떤가요?'}
]

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

trajectory += response.output

for output in response.output:
    if output.type == "function_call":
        print("✅ Tool Call Maded!")
        if output.name == "get_weather":
            # execute tool function
            tool_result = get_weather(**json.loads(output.arguments))
            print("✅ Tool Function Executed!")
            print(f"     📍 Result : {tool_result}")

            # add tool result to trajectory
            trajectory.append({
                'type' : 'function_call_output',
                'call_id' : output.call_id,
                'output' : json.dumps({
                    "weather_info" : tool_result
                })
            })           

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

for output in response.output:
    if output.type == "message":
        for content in output.content:
            if content.type == "output_text":
                print(content.text)

✅ Tool Call Maded!
✅ Tool Function Executed!
     📍 Result : Weather in Seoul, KR is 10 celsius.
서울의 현재 날씨는 섭씨 10도입니다.


In [24]:
trajectory

[{'role': 'system', 'content': '당신의 이름은 ReflACE Agent 입니다.'},
 {'role': 'user', 'content': '현재 서울의 날씨는 어떤가요?'},
 ResponseFunctionToolCall(arguments='{"location":"Seoul, KR","units":"celsius"}', call_id='call_gtDiWI8sjnkqFxWcZddRODkT', name='get_weather', type='function_call', id='fc_02d448c8cd97ffb900694387d9003c81908123434c62c606c3', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_gtDiWI8sjnkqFxWcZddRODkT',
  'output': '{"weather_info": "Weather in Seoul, KR is 10 celsius."}'}]

## AppWorld

### ReAct Agent

In [3]:
env = AppWorldEnv(task_type="train")

env.set_env(experiment_name="sample")

with open(os.path.join('src','prompt','react','system_prompt_oneshot.txt'), 'r', encoding='utf-8') as f:
    system_prompt = f.read()

llm_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    system_prompt=system_prompt
)

react_agent = ReActAgent(llm_client=llm_client)

result = react_agent.run(env=env,max_steps=30)

In [6]:
evaluation = env.env.evaluate()

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 2

Num Failed Tests : 0

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

assert answers match.

>> Passed Requirement

assert no model changes.

────────────────────────────────────────────────────── Fails ──────────────────────────────────────────────────────

None

In [20]:
print(evaluation.pass_count)
print(evaluation.fail_count)
print(evaluation.total_count)
print(evaluation.pass_percentage)

2
0
2
100.0


### Reflexion Agent

In [17]:
env = AppWorldEnv(task_type="train")
env.set_env(experiment_name="sample")

with open(os.path.join('src','prompt','react','system_prompt_oneshot.txt'), 'r', encoding='utf-8') as f:
    actor_system_prompt = f.read()

with open(os.path.join('src','prompt','reflexion','reflector_system_prompt.txt'), 'r', encoding='utf-8') as f:
    reflector_system_prompt = f.read()

actor_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    system_prompt=actor_system_prompt
)
reflector_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    system_prompt=reflector_system_prompt
)

reflexion_agent = ReflexionAgent(
    action_module_client=actor_client,
    reflection_module_client=reflector_client
)

result = reflexion_agent.run(env=env,max_retries=3)

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 2

Num Failed Tests : 0

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

assert answers match.

>> Passed Requirement

assert no model changes.

────────────────────────────────────────────────────── Fails ──────────────────────────────────────────────────────

None

### Evaluate Performance

> ReAct Agent Performance Report (First 5 Environments in 90 train environments)
- ✅ Success Task : 14
- ❌ Failed Task : 6
- 📊 Task Success Rate : 70.00%

> Reflexion Agent Performance Report (First 5 Environments in 90 train environments)
- ✅ Success Task : 18
- ❌ Failed Task : 2
- 📊 Task Success Rate : 90.00%

#### ReAct

In [ ]:
env = AppWorldEnv(task_type="train")
env.set_env()

with open(os.path.join('src','prompt','react','system_prompt_oneshot.txt'), 'r', encoding='utf-8') as f:
    system_prompt = f.read()

llm_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    system_prompt=system_prompt
)

react_agent = ReActAgent(llm_client=llm_client)

react_performance = evaluate_agent(agent=react_agent, env=env, max_task=5)

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 2

Num Failed Tests : 0

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

assert song_ids of the updated song reviews match private_data.to_update_review_song_ids (ignore_order)

```python
with test(
    """
    assert song_ids of the updated song reviews match private_data.to_update_review_song_ids (ignore_order)
    """
):
    updated_review_song_ids = list_of(updated_song_reviews, "song_id")
    test.case(
        updated_review_song_ids, "==", private_data.to_update_review_song_ids, ignore_order=True
```
----------
AssertionError:  [] == [12, 247]

In right but not left:
[12, 247]

>> Failed Requirement

assert all updated_review_song_ids have rating of public_data.target_rating.

```python
with test(
    """
    assert all updated_review_song_ids have rating of public_data.target_rating.
    """
):
```
----------
AssertionError: The left modifier is 'all' with allow_left_empty=False, but left operand is empty.

In [ ]:
print("ReAct Agent Performance Report")
print(f"    📝 Total Task : {react_performance['total_cnt']}")
print(f"    ✅ Success Task : {react_performance['success_cnt']}")
print(f"    ❌ Failed Task : {react_performance['failed_cnt']}")
print(f"    📊 Task Success Rate : {react_performance['success_rate'] * 100:.2f}%")

ReAct Agent Performance Report
    ✅ Success Task : 14
    ❌ Failed Task : 6
    📊 Task Success Rate : 70.00%


#### Reflexion

In [ ]:
env = AppWorldEnv(task_type="train")
env.set_env(experiment_name="sample")

with open(os.path.join('src','prompt','react','system_prompt_oneshot.txt'), 'r', encoding='utf-8') as f:
    actor_system_prompt = f.read()

with open(os.path.join('src','prompt','reflexion','reflector_system_prompt.txt'), 'r', encoding='utf-8') as f:
    reflector_system_prompt = f.read()

actor_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    system_prompt=actor_system_prompt
)
reflector_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.0,
    system_prompt=reflector_system_prompt
)

reflexion_agent = ReflexionAgent(
    action_module_client=actor_client,
    reflection_module_client=reflector_client
)

reflexion_performance = evaluate_agent(reflexion_agent, env, max_task=5)

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 2

Num Failed Tests : 0

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

assert song_ids of the updated song reviews match private_data.to_update_review_song_ids (ignore_order)

```python
with test(
    """
    assert song_ids of the updated song reviews match private_data.to_update_review_song_ids (ignore_order)
    """
):
    updated_review_song_ids = list_of(updated_song_reviews, "song_id")
    test.case(
        updated_review_song_ids, "==", private_data.to_update_review_song_ids, ignore_order=True
```
----------
AssertionError:  [190, 231, 265] == [12, 247]

In left but not right:
[190, 231, 265]

In right but not left:
[12, 247]

Original values:
[265, 231, 190] == [12, 247]

>> Failed Requirement

assert song_ids of the added song reviews match private_data.to_add_review_song_ids (ignore_order)

```python
with test(
    """
    assert song_ids of the added song reviews match private_data.to_add_review_song_ids (ignore_order)
    """
):
    added_review_song_ids = list_of(added_song_reviews, "song_id")
    test.case(
        added_review_song_ids, "==", private_data.to_add_review_song_ids, ignore_order=True
```
----------
AssertionError:  [10, 31, 173, 202, 304, 310] == [31, 173, 202, 304, 310]

In left but not right:
[10]

Original values:
[304, 173, 310, 202, 31, 10] == [202, 173, 304, 310, 31]

In [18]:
print("Reflexion Agent Performance Report")
print(f"    ✅ Success Task : {success_cnt}")
print(f"    ❌ Failed Task : {fail_cnt}")
print(f"    📊 Task Success Rate : {success_cnt / (success_cnt + fail_cnt) * 100:.2f}%")

Reflexion Agent Performance Report
    ✅ Success Task : 18
    ❌ Failed Task : 2
    📊 Task Success Rate : 90.00%
